In [237]:
import gurobi_logtools as glt
import pandas as pd
import plotly.express as px
import plotly.subplots as sp

In [238]:
F = ["RU", "SBL", "SBM", "SU"]
N = ["10", "15", "20", "25"]
P = ["111", "110", "011", "010"]
O = ["StripPacking"]

Params = [(f,p,o,n)  for f in F  for p in P  for o in O  for n in N]
Files = [f + "-" + p + "-" + o + "-" + n + ".log"  for (f,p,o,n) in Params]
Results = glt.parse(Files)
Info = pd.DataFrame({'Formulation': [f    for (f,p,o,n) in Params], 'N': [n   for (f,p,o,n) in Params]})
Summary = Results.summary().join(Info)
Progress = Results.progress("nodelog")
fs = []
cs = []
hs = []
ns = []
for index, row in Progress.iterrows():
    file = row['LogFilePath']
    dash = file.index('-')
    fs.append(file[0:dash])
    cs.append('(' + file[dash+1] + ',' + file[dash+3] +')')
    ns.append(file[-6:-4])
Progress['Formulation'] = fs
Progress['(C,H)'] = cs
Progress['N'] = ns

In [290]:
Filter = Progress[Progress['LogFilePath'].str.contains('SU') & Progress['LogFilePath'].str.contains('-20')]
glt.plot(Filter, type='line', color='(C,H)')

interactive(children=(Dropdown(description='x', index=9, options=('CurrentNode', 'RemainingNodes', 'Obj', 'Dep…

In [287]:
Filter = Progress[Progress['LogFilePath'].str.contains('SU') & Progress['LogFilePath'].str.contains('-20')]

fig1 = px.line(Filter, x="Time", y="Gap", color="(C,H)")
fig2 = px.line(Filter, x="Time", y="ItPerNode", color="(C,H)")



fig = sp.make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05)



for trace in range(len(fig1["data"])):
    fig.append_trace(fig1["data"][trace], row=1, col=1)
for trace in range(len(fig2["data"])):
    fig2["data"][trace]['showlegend'] = False
    fig.append_trace(fig2["data"][trace], row=2, col=1)
fig['layout']['legend']['title']['text'] = fig1['layout']['legend']['title']['text']
#fig['layout']['xaxis2']['title']['text'] = fig1['layout']['xaxis']['title']['text']
#fig['layout']['yaxis1']['title']['text'] = fig1['layout']['yaxis']['title']['text']
#fig['layout']['yaxis2']['title']['text'] = fig2['layout']['yaxis']['title']['text']
fig['layout']['xaxis2']['title']['text'] = 'Time (seconds)'
fig['layout']['yaxis1']['title']['text'] = 'Optimality Gap (%)'
fig['layout']['yaxis2']['title']['text'] = 'Iterations/Node'

fig['layout'].update(margin=dict(l=20,r=20,b=20,t=40))
fig.update_layout(yaxis2 = dict(range=[5, 95]))

fig

In [301]:
Filter = Progress[Progress['Formulation'].isin(['SU']) 
                  & Progress['N'].isin(['15']) 
                  & Progress['(C,H)'].isin(['(0,0)','(1,1)'])]

fig1 = px.line(Filter, x="Time", y=["Incumbent","BestBd"], color="(C,H)")

fig1